In [ ]:

from bs4 import BeautifulSoup # For HTML parsing
import requests # Website connections
from time import sleep # To prevent overwhelming the server between connections
from collections import Counter # Keep track of our term counts
import pandas as pd # For converting results to a dataframe and bar chart plots
import json # For parsing json
%matplotlib inline
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
driver = webdriver.Chrome('C:/Users/Minglin886/Downloads/chromedriver_win32/chromedriver.exe')

In [ ]:
driver1 = webdriver.Chrome('C:/Users/Minglin886/Downloads/chromedriver_win32/chromedriver.exe')

In [ ]:
driver.get("https://prod.education.gouv.qc.ca/gdunojrecherche/rechercheOrganisme.do?methode=afficherResultatBase&typeRecherche=base")

In [ ]:
driver.find_element_by_xpath("//select[@name='codeOrdreEnseignementAppartenance']/option[@value='educationPPS']").click()

In [ ]:
driver.find_element_by_id("ordreDispensePrimaire").click()
driver.find_element_by_id("ordreDispenseSecondaire").click()
driver.find_element_by_id("proprieteVide7").click()

In [ ]:
for i in range(1,182):
    if (i==1):
        list_t=[]
    #each page
    source= driver.page_source
    soup = BeautifulSoup(source)
    oddlink=[tag.a.get('href') for tag in soup.find_all('tr',class_='couleurpair')]
    evenlink=[tag.a.get('href') for tag in soup.find_all('tr',class_='couleurimpair')]
    x=0
    y=0
    for odd in oddlink:
        driver1.get(f'https://prod.education.gouv.qc.ca/gdunojrecherche/{odd}')
        response= driver1.page_source
        soupodd= BeautifulSoup(response)
        step1= soupodd.find('table',class_='tableaffichageVerticalIE')
        seperated=step1.find_all('td')[1].get_text(separator='|br|', strip=True).split('|br|')
        address=" ".join(seperated[:-1])
        citypostcode=seperated[-1].split('\xa0\xa0')
        city=citypostcode[0]
        postcode=citypostcode[1]
        name= soupodd.find('h2', style="background-color: #EEEBF2;").text.strip()
        clas= [tag.get_text(strip= True) for tag in soupodd.find_all('td',class_='couleurCellule')]           
        school_board= clas[-5]
        level=clas[-4]
        code=clas[-1]
        list_t.append((address,city,postcode,name,school_board,level,code))
        x+=1
        print(f'odd{x}')
    for even in evenlink:
        driver1.get(f'https://prod.education.gouv.qc.ca/gdunojrecherche/{even}')
        response= driver1.page_source
        soupeven= BeautifulSoup(response)
        step1= soupeven.find('table',class_='tableaffichageVerticalIE')
        seperated=step1.find_all('td')[1].get_text(separator='|br|', strip=True).split('|br|')
        address=" ".join(seperated[:-1])
        citypostcode=seperated[-1].split('\xa0\xa0')
        city=citypostcode[0]
        postcode=citypostcode[1]
        name= soupeven.find('h2', style="background-color: #EEEBF2;").text.strip()
        clas= [tag.get_text(strip= True) for tag in soupeven.find_all('td',class_='couleurCellule')]
        school_board= clas[-5]
        level=clas[-4]
        code=clas[-1]
        list_t.append((address,city,postcode,name,school_board,level,code))
        y+=1
        print(f'even{y}')
    if ((i%20)==0) or (i==len(range(1,182))):##i==181-----break
        
        df= pd.DataFrame(list_t, columns=['address','city','postcode','name','school_board','level','code'])
        # to csv 
        df.to_csv(f'test_q{i}.csv', sep=',',index=False,encoding='utf-8-sig')
        if i==len(range(1,182)):
            break
        list_t=[]
    img= driver.find_element_by_xpath("//*[@id='bt_suivant']")
    img.find_element_by_xpath('..').click()
    sleep(2)
    
    